# Limpieza: Predicción de la presencia altos niveles de PM10

En este notebook vamos a tomar los datasets de contaminantes y metereología obtenidos de http://www.aire.cdmx.gob.mx/default.php

Juntaremos los dataframes con una PivotTable y las agruparemos por el momento de la medición

In [1]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('fivethirtyeight')
import pandas as pd
import matplotlib
import seaborn as sns

from datetime import timedelta

Definimos unas funciones para convertir las fechas de DD-MM-AAAA a AAAA-MM-DD y para convertir las horas 24:00 en 00:00:

In [2]:
def time_converter(x):
    x0 = x.split(" ")[0]
    x0 = x0.split("/")
    x1 = x.split(" ")[1]
    if x1[:].endswith("24:00"):
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" 00:00"
    else:
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" "+ x1[:]

In [3]:
def time_converter_guion(x):
    x0 = x.split(" ")[0]
    x0 = x0.split("-")
    x1 = x.split(" ")[1]
    if x1[:].endswith("24:00"):
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" 00:00"
    else:
        return x0[2]+"-"+x0[1]+"-"+x0[0]+" "+ x1[:]

In [4]:
def time_converter_date(x):
    x0 = x.split("/")
    return x0[2]+"-"+x0[1]+"-"+x0[0]

## Carga de los conjuntos de datos   <a class="anchor" id="limpieza-bullet"></a>

Definamos el año a limpiar:

In [779]:
a="2000"

Cargamos los datos

### Metereología

In [780]:
met_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Metereología/meteorología_' + a + ".CSV"),header=10)
met_2018 = met_2018.dropna(how='any')
met_2018 = met_2018.drop(['unit'], axis=1)

### Contaminantes

In [781]:
cont_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/Contaminantes/contaminantes_'+ a +'.CSV'),header=10)
cont_2018 = cont_2018.dropna(how='any')
cont_2018 = cont_2018.drop(['unit'], axis=1)

In [782]:
cont_2018.head(2)

,date,cve_station,cve_parameter,value
0,01/01/2000 01:00,LAG,CO,2.3
1,01/01/2000 01:00,TAC,CO,1.0


### Radiación UVA

In [783]:
radA_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/UV/UVA/UVA_'+ a +'.csv'),header=8)
radA_2018 = radA_2018.fillna(0)
radA_2018 = radA_2018.drop(['unit'], axis=1)

### Radiación UVB

In [784]:
radB_2018 = pd.read_csv(str('/Users/danielbustillos/Documents/servicio/Contaminación PM10/UV/UVB/UVB_'+ a +'.csv'),header=8)
radB_2018 = radB_2018.fillna(0)
radB_2018 = radB_2018.drop(['unit'], axis=1)

## Append Radiación

Juntamos los df. de UVA y UVBm

In [785]:
rad_2018 = radA_2018.append(radB_2018, ignore_index=True)

In [786]:
rad_2018.head(2)

,Date,cve_station,parameter,value
0,01-01-2000 01:00,HAN,UVA,0.0
1,01-01-2000 01:00,MER,UVA,0.0


# Limpieza paso a paso

### Pres

Nos quedamos con las columnas indicadas y reseteamos el index:

### Contaminantes

In [787]:
cont_2018 = cont_2018.rename(columns={'Date': 'date', 'cve_station': 'id_station','cve_parameter': 'id_parameter'})

In [788]:
cont_ACO = cont_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
cont_ACO = cont_ACO.reset_index(drop=False)
cont_ACO = cont_ACO[["date","id_station","id_parameter","value"]]

In [789]:
cont_ACO.head(2)

,date,id_station,id_parameter,value
0,01/01/2000 01:00,LAG,CO,2.3
1,01/01/2000 01:00,TAC,CO,1.0


### Metereologia

In [790]:
met_2018 = met_2018.rename(columns={'Date': 'date', 'cve_station': 'id_station','cve_parameter': 'id_parameter'})

In [791]:
met_2018.head()

,date,id_station,id_parameter,value
3,01/01/2000 01:00,TLA,RH,38.0
4,01/01/2000 01:00,XAL,RH,51.0
5,01/01/2000 01:00,MER,RH,47.0
7,01/01/2000 01:00,CES,RH,36.0
8,01/01/2000 01:00,PLA,RH,32.0


In [792]:
met_ACO = met_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
met_ACO = met_ACO.reset_index(drop=False)
met_ACO = met_ACO[["date","id_station","id_parameter","value"]]

### Radiación

In [793]:
rad_2018 = rad_2018.rename(columns={'Date': 'date', 'cve_station': 'id_station','parameter': 'id_parameter'})

In [794]:
rad_ACO = rad_2018
#cont_ACO = cont_ACO[(cont_ACO["id_parameter"] == "PM10")]
rad_ACO = rad_ACO.reset_index(drop=False)
rad_ACO = rad_ACO[["date","id_station","id_parameter","value"]]

In [795]:
rad_ACO.head()

,date,id_station,id_parameter,value
0,01-01-2000 01:00,HAN,UVA,0.0
1,01-01-2000 01:00,MER,UVA,0.0
2,01-01-2000 01:00,MON,UVA,0.0
3,01-01-2000 01:00,PED,UVA,0.0
4,01-01-2000 01:00,SAG,UVA,0.0


## Pivot_Table <a class="anchor" id="pivot-bullet"></a>

### Pre

### Contaminantes

In [796]:
cont_ACO_hour.head()

id_parameter,CO,NO2,NOX,O3,PM10,SO2,id_station,date
0,8.4,NaN,NaN,NaN,NaN,31.0,ARA,2001-01-01 01:00:00
1,NaN,23.0,45.0,NaN,NaN,NaN,ATI,2001-01-01 01:00:00
2,1.1,20.0,43.0,13.0,NaN,13.0,AZC,2001-01-01 01:00:00
3,1.8,21.0,39.0,3.0,NaN,9.0,BJU,2001-01-01 01:00:00
4,1.6,17.0,27.0,12.0,66.0,17.0,CES,2001-01-01 01:00:00


In [797]:
cont_ACO.id_parameter.unique()

array(['CO', 'NO2', 'NOX', 'O3', 'PM10', 'SO2'], dtype=object)

In [798]:
cont_ACO_hour = pd.pivot_table(cont_ACO,index=["date","id_station"],columns=["id_parameter"])
cont_ACO_hour = cont_ACO_hour.reset_index(drop=False)
cont_ACO_hour.columns = cont_ACO_hour.columns.droplevel()
cont_ACO_hour["id_station"] = cont_ACO_hour.iloc[:,1]
cont_ACO_hour["date"] = cont_ACO_hour.iloc[:,0]
cont_ACO_hour = cont_ACO_hour.drop([""],axis=1)
cont_ACO_hour['date'] = cont_ACO_hour.apply(lambda row: time_converter(row['date']), axis=1) 
cont_ACO_hour['date'] =  pd.to_datetime(cont_ACO_hour['date'], format='%Y-%m-%d %H:%M')
#cont_ACO_hour = cont_ACO_hour[[ "date" ,"id_station",'CO', 'NO', 'NO2', 'NOX', 'O3', 'PM2.5', 'PMCO', 'SO2','PM10']]

In [799]:
#cont_ACO_hour['date'] = cont_ACO_hour["date"].str[0:10]

In [800]:
cont_ACO_hour.head()

id_parameter,CO,NO2,NOX,O3,PM10,SO2,id_station,date
0,5.3,NaN,NaN,NaN,NaN,26.0,ARA,2000-01-01 01:00:00
1,1.5,20.0,28.0,NaN,NaN,11.0,ATI,2000-01-01 01:00:00
2,3.5,NaN,NaN,6.0,NaN,32.0,AZC,2000-01-01 01:00:00
3,2.3,27.0,97.0,4.0,NaN,15.0,BJU,2000-01-01 01:00:00
4,2.6,56.0,110.0,8.0,69.0,NaN,CES,2000-01-01 01:00:00


### Metereología

In [801]:
met_ACO_hour = pd.pivot_table(met_ACO,index=["date","id_station"],columns=["id_parameter"])
met_ACO_hour = met_ACO_hour.reset_index(drop=False)
met_ACO_hour.columns = met_ACO_hour.columns.droplevel()
met_ACO_hour["id_station"] = met_ACO_hour.iloc[:,1]
met_ACO_hour["date"] = met_ACO_hour.iloc[:,0]
met_ACO_hour = met_ACO_hour.drop([""],axis=1)
met_ACO_hour['date'] = met_ACO_hour.apply(lambda row: time_converter(row['date']), axis=1) 
met_ACO_hour['date'] =  pd.to_datetime(met_ACO_hour['date'], format='%Y-%m-%d %H:%M')
#met_ACO_hour = met_ACO_hour[["date","id_station","RH","TMP","WSP","WDR"]]

### Radiación

In [802]:
rad_ACO.head()

,date,id_station,id_parameter,value
0,01-01-2000 01:00,HAN,UVA,0.0
1,01-01-2000 01:00,MER,UVA,0.0
2,01-01-2000 01:00,MON,UVA,0.0
3,01-01-2000 01:00,PED,UVA,0.0
4,01-01-2000 01:00,SAG,UVA,0.0


In [803]:
rad_ACO = pd.pivot_table(rad_ACO,index=["date","id_station"],columns=["id_parameter"])
rad_ACO = rad_ACO.reset_index(drop=False)
rad_ACO.columns = rad_ACO.columns.droplevel()
rad_ACO["id_station"] = rad_ACO.iloc[:,1]
rad_ACO["date"] = rad_ACO.iloc[:,0]
rad_ACO = rad_ACO.drop([""],axis=1)
rad_ACO['date'] = rad_ACO.apply(lambda row: time_converter_guion(row['date']), axis=1)
rad_ACO['date'] =  pd.to_datetime(rad_ACO['date'], format='%Y-%m-%d %H:%M')
rad_ACO = rad_ACO[[ "date","id_station",'UVA',"UVB"]]
rad_ACO = rad_ACO.dropna(how='any')

# Merge de Dataframes   <a class="anchor" id="merge-bullet"></a>

Por hora: no es posible añadir precipitación ya que de origen viene por día.

Juntamos los dataframes:

In [804]:
cont_ACO_hour['date'] =  pd.to_datetime(cont_ACO_hour['date'], format='%Y-%m-%d %H:%M')
met_ACO_hour['date'] =  pd.to_datetime(met_ACO_hour['date'], format='%Y-%m-%d %H:%M')

In [805]:
data_hour_merge = pd.merge(cont_ACO_hour, met_ACO_hour, on=["date","id_station"],how="outer")

In [806]:
data_hour_merge = pd.merge(data_hour_merge, rad_ACO, on=["date","id_station"],how="outer")

Eliminamos los NAN:

Definimos las columnas hora, dia, mes

Cambiamos el nombre de date a fecha:

In [807]:
data_hour_merge = data_hour_merge.rename(columns={'date': 'fecha'})

In [808]:
#data_hour_merge = data_hour_merge.drop(['date'],axis=0)

Exportamos:

In [809]:
#data_hour_merge = data_hour_merge.dropna()

In [810]:
data_hour_merge = data_hour_merge.dropna(subset=['O3'])

In [811]:
data_hour_merge['hora'] = data_hour_merge['fecha'].astype(str).str[10:13]

In [812]:
data_hour_merge['dia'] = data_hour_merge['fecha'].astype(str).str[8:10]

In [813]:
data_hour_merge['mes'] = data_hour_merge['fecha'].astype(str).str[5:7]

In [814]:
data_hour_merge['fecha'] = data_hour_merge['fecha'].astype(str).str[0:10]

In [815]:
data_hour_merge.to_csv(str("/Users/danielbustillos/Documents/servicio/Contaminación PM10/Outputs/por_hora/cont_hora" + a + ".csv"))